### Modelos

- Excecução de cross section
- Efeitos fixos
- Efeitos aleatórios



In [1]:
import pandas as pd
import numpy as np

# Modelo
import  statsmodels.api as sm 
from linearmodels.panel import PanelOLS

In [2]:
panel_data = pd.read_parquet('../Dados/pnad_covid/pnad_covid_balanced.parquet')

In [3]:
panel_data.dropna(inplace=True)

In [4]:
panel_data.reset_index(inplace=True)

In [5]:
#for every portfolio, fill mising values with first value of sigla_uf



panel_data['C013'] = panel_data['C013'].fillna(method='bfill')

panel_data['A003'] = panel_data['A003'].fillna(method='bfill')

panel_data['A004'] = panel_data['A004'].fillna(method='bfill')

panel_data['A005'] = panel_data['A005'].fillna(method='bfill')



In [6]:
#for each domicilio_id fill missing values with first value diferent of NaN in column

panel_data['A003'] = panel_data.groupby('domicilio_id')['A003'].transform(lambda x: x.fillna(method='bfill'))

### Regressão

In [7]:
panel_data.set_index(['domicilio_id', 'semana_dt'], inplace=True)

In [8]:
#Variáveis explicativas do modelo
# Adicionando uma constante
exog_variables = panel_data.drop('C013', axis=1)

In [9]:
panel_data

C013    A003    A004  \
domicilio_id            semana_dt                        
1100000161Mulher221974  18         Não  Mulher   Parda   
                        22         Não  Mulher   Parda   
                        26         Não  Mulher   Parda   
                        30         Não  Mulher   Parda   
                        34         Não  Mulher   Parda   
...                                ...     ...     ...   
53001448813Homem2331978 42         Sim   Homem  Branca   
53001448813Mulher721980 42         Não  Mulher  Branca   
5300193734Mulher2081982 44         Sim  Mulher   Parda   
5300317391Homem18112000 44         Sim   Homem   Parda   
5300317391Homem4111997  44         Não   Homem   Parda   

                                                                   A005  \
domicilio_id            semana_dt                                         
1100000161Mulher221974  18                               Médio completo   
                        22                               Médio completo   
                        26                               Médio completo   
                        30                               Médio completo   
                        34                               Médio completo   
...                                                                 ...   
53001448813Homem2331978 42         Pós-graduação, mestrado ou doutorado   
53001448813Mulher721980 42         Pós-graduação, mestrado ou doutorado   
5300193734Mulher2081982 44                            Superior completo   
5300317391Homem18112000 44                          Superior incompleto   
5300317391Homem4111997  44                               Médio completo   

                                  formalidade semana  
domicilio_id            semana_dt                     
1100000161Mulher221974  18             Formal     24  
                        22             Formal     24  
                        26             Formal     24  
                        30             Formal     24  
                        34             Formal     24  
...                                       ...    ...  
53001448813Homem2331978 42             Formal     20  
53001448813Mulher721980 42             Formal     20  
5300193734Mulher2081982 44             Formal     20  
5300317391Homem18112000 44           Informal     20  
5300317391Homem4111997  44           Informal     20  

[977444 rows x 6 columns]

In [11]:
pd.get_dummies(exog_variables[['A003','A004', 'A005', 'formalidade', 'semana']]).drop(['A003_Homem', 'A004_Brancas e Amarelas', 'A005_Sem instrução', 'formalidade_Informal'], axis=1)

KeyError: "['A004_Brancas e Amarelas'] not found in axis"

In [12]:
pd.get_dummies(exog_variables[['A003','A004', 'A005', 'formalidade']])[['A003_Mulher', 'A004_Pretas e Pardas', 'formalidade_Formal', 'A005_Fundamental completo e Médio incompleto', 'A005_Médio completo e Superior incompleto']]

KeyError: "['A004_Pretas e Pardas', 'A005_Fundamental completo e Médio incompleto', 'A005_Médio completo e Superior incompleto'] not in index"

In [13]:
exog_variables = pd.get_dummies(exog_variables).drop(['A003_Homem', 'A004_Brancas e Amarelas', 'A005_Sem instrução', 'formalidade_Informal'], axis=1)

KeyError: "['A004_Brancas e Amarelas'] not found in axis"

In [14]:
exog_variables.shape

(977444, 5)

In [17]:
exog_variables.drop('semana', axis=1, inplace=True)

In [18]:
endog_variable = pd.get_dummies(panel_data['C013'])['Sim']

In [19]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Selecionar apenas as variáveis explicativas numéricas
exog_numeric = exog_variables.select_dtypes(include=np.number)

# Calcular os VIFs
vifs = pd.DataFrame()
vifs["Variável"] = exog_numeric.columns
vifs["VIF"] = [variance_inflation_factor(exog_numeric.values, i) for i in range(exog_numeric.shape[1])]

print(vifs)

Empty DataFrame
Columns: [Variável, VIF]
Index: []


In [21]:
# Estimating the fixed effects model
f_effects_model = PanelOLS(dependent = endog_variable,  # Variável endógena (Y)
                           exog = exog_variables,  # Variáveis exógenas (X matrix)
                           check_rank=False,  # Checar se a matriz X é de posto completo
                           entity_effects = True,  # True -> Efeitos fixos
                           drop_absorbed = True,# Remover variáveis absorvidas
                           time_effects=True)  

# Ajustar o modelo aos dados
f_effects_results = f_effects_model.fit()

/tmp/ipykernel_8926/1146912806.py:10: MemoryWarning: Using low-memory algorithm to estimate two-way model. Explicitly set low_memory=True to silence this message.  Set low_memory=False to use the standard algorithm that creates dummy variables for the smaller of the number of entities or number of time periods.
  f_effects_results = f_effects_model.fit()
/tmp/ipykernel_8926/1146912806.py:10: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

A003.Homem

  f_effects_results = f_effects_model.fit()


In [22]:
f_effects_results = f_effects_model.fit()

/tmp/ipykernel_8926/3205855132.py:1: MemoryWarning: Using low-memory algorithm to estimate two-way model. Explicitly set low_memory=True to silence this message.  Set low_memory=False to use the standard algorithm that creates dummy variables for the smaller of the number of entities or number of time periods.
  f_effects_results = f_effects_model.fit()


In [23]:
f_effects_results.summary

Dep. Variable:,Sim,R-squared:,0.0003
Estimator:,PanelOLS,R-squared (Between):,0.0489
No. Observations:,977444,R-squared (Within):,0.0002
Date:,"Wed, Sep 06 2023",R-squared (Overall):,0.0418
Time:,17:48:48,Log-likelihood,6.266e+05
Cov. Estimator:,Unadjusted,,
,,F-statistic:,13.830
Entities:,210843,P-value,0.0000
Avg Obs:,4.6359,Distribution:,"F(14,766560)"
Min Obs:,1.0000,,
Max Obs:,14.000,F-statistic (robust):,13.830


---

Efeito do 2º choque - Flexibilização